In [1]:
# import codecs
# f = codecs.open("test_data.txt", 'r', 'utf-8') #記得以utf-8開啟文字檔
# location = f.read()
# f.close()
# print(location)

import csv
import os
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
# 操作 browser 的 API
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

# ChromeDriver 的下載管理工具
from webdriver_manager.chrome import ChromeDriverManager

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

from selenium.common.exceptions import ElementNotInteractableException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By


# 強制等待 (執行期間休息一下)
from time import sleep

# with open(os.path.join("科技執法總清單20230611.csv"), "r", newline="", encoding="utf-8") as file:

# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")         #最大化視窗
my_options.add_argument("--incognito")               #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消 chrome 推播通知
my_options.add_argument("--lang=zh-TW")  #設定為正體中文
my_options.add_argument('--user-agent=Mozilla/5.0 (Macintosh Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36')


In [2]:
# 開啟網頁
chrome_path = 'C:\Program Files\Google\Chrome\Application\Chrome.exe'
options = webdriver.ChromeOptions()
options.binary_location = chrome_path
driver = webdriver.Chrome(options=options)



In [3]:
# 將剛開起的空網頁導向要去的網頁
def get_Latitude_And_Longitude(url):
    driver.get(url)

In [4]:
# 抓經緯度的正規表達式
all = r"@(-?\d+\.\d+),(-?\d+\.\d+),"

In [7]:
Traffic_Enforcement_Camera = pd.read_csv("地址轉座標.csv", header=0)
for i in range (len(Traffic_Enforcement_Camera['縣市'])):
    address = "{}{}".format(Traffic_Enforcement_Camera['縣市'][i],Traffic_Enforcement_Camera['地點'][i])
    url = "https://www.google.com/maps/place/{}".format(address)
    get_Latitude_And_Longitude(url)
    sleep(6)
    title = driver.title
    if (title != "Google 地圖"):
        current_url = driver.current_url
        match = re.search(all, current_url)
        match_Latitude = match.group(2)
        match_Longitude = match.group(1)
        Traffic_Enforcement_Camera.loc[i, '緯度'] = match_Longitude
        Traffic_Enforcement_Camera.loc[i, '經度'] = match_Latitude
        print("第{}筆座標轉換成功,{},{}".format(i,match_Longitude,match_Latitude))
        if (i%10 == 0):
            Traffic_Enforcement_Camera.to_csv("地址轉座標.csv", index=False, encoding="utf_8_sig")
    else:
        print("{}座標轉換失敗,索引值為{}".format(address,i))
    

Traffic_Enforcement_Camera.to_csv("地址轉座標.csv", index=False, encoding="utf_8_sig")

第0筆座標轉換成功,24.9507874,121.3499414
第1筆座標轉換成功,24.9677016,121.3331537
第2筆座標轉換成功,25.0254362,121.4672131
第3筆座標轉換成功,25.1422146,121.3899541
第4筆座標轉換成功,25.1307022,121.3919947
新北市八里區臺15線中山路三段383號座標轉換失敗,索引值為5
第6筆座標轉換成功,25.1253012,121.4532319
第7筆座標轉換成功,25.1526832,121.440581
第8筆座標轉換成功,25.1594386,121.425125
第9筆座標轉換成功,25.0820727,121.4987089
第10筆座標轉換成功,25.0882332,121.4939919
第11筆座標轉換成功,25.0629015,121.5035673
第12筆座標轉換成功,25.0587145,121.4894253
第13筆座標轉換成功,24.9202954,121.3693379
第14筆座標轉換成功,24.912414,121.3657106
第15筆座標轉換成功,24.8981171,121.3376371
第16筆座標轉換成功,24.934322,121.389644
第17筆座標轉換成功,24.9430838,121.4012911
第18筆座標轉換成功,24.9259777,121.4413489
第19筆座標轉換成功,24.9319962,121.4546451
第20筆座標轉換成功,24.9028487,121.3735001
第21筆座標轉換成功,24.9337739,121.403965
第22筆座標轉換成功,24.929729,121.4109782
第23筆座標轉換成功,24.9787792,121.4510238
第24筆座標轉換成功,24.9830515,121.4395552
第25筆座標轉換成功,24.9707949,121.4427159
第26筆座標轉換成功,25.010059,121.4802001
第27筆座標轉換成功,25.0859043,121.4433137
第28筆座標轉換成功,25.0868721,121.4433139
第29筆座標轉換成功,25.0900393,121.4464149

In [5]:
i ="https://www.google.com/maps/place//@25.0029159,121.5255809,15z/data=!3m1!4b1?entry=ttu"
get_Latitude_And_Longitude(i)
sleep(4)
current_url = driver.current_url

 # 將讀取的csv去除掉前兩行存在rows變數裡
# reader = csv.reader(file)
# rows = list(reader)


#         # 將觀測站的ID塞到第一列
#         for row in rows[0:]:
#             row.insert(0, f"{match_station_ID}")

#         # 將日期塞到第二列
#         for row in rows[0:]:
#             row.insert(1, f"{match_whole_date}")
# print(current_url)

# match = re.search(all, current_url)
# print(match.group(1))
# print(match.group(2))
title = driver.title
print(f"Title of the web page is: {title}")


Title of the web page is: Google 地圖
